In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import os
import requests
import json
load_dotenv(override=True)

In [ ]:
openai = OpenAI()

In [ ]:
telegram_bot_token = os.getenv("TELEGRAM_BOT_TOKEN")
group_chat_id = os.getenv("GROUP_CHAT_ID")

The following function is used to sendd the message to user telegram.

In [ ]:
def send_message(message: str):
    url = f"https://api.telegram.org/bot{telegram_bot_token}/sendMessage"
    payload = {
        "chat_id": group_chat_id,
        "text": message,
        "parse_mode": "Markdown",
    }
    response = requests.post(url, data=payload)
    return response.json()

The following function is used to collect the user information.

In [ ]:
def record_user_details(email, name = "Name not provided", notes = "No additional notes"):
    details = f"""
    **New User Details:**
    - Name: {name}
    - Email: {email}
    - Notes: {notes}
    """
    send_message(details)
    return {"recorded": "ok"}

In [ ]:
def record_unknown_questions(question):
    message = f"**Unknown Question Asked:**\n{question}"
    send_message(message)
    return {"recorded": "ok"}

In [ ]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is in interested being in touch and provided their email",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The user's email address"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            },
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [ ]:
recorde_unknown_questions_json = {
    "name": "record_unknown_questions",
    "description": "Use this tool to record any questions that the chatbot was unable to answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that the chatbot was unable to answer"
            }
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": record_user_details_json}, 
         {"type": "function", "function": recorde_unknown_questions_json}]

In [ ]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        print(f"tool name: {tool_name}")
        print(f"arguments: {tool_call.function.arguments}")
        arguments = json.loads(tool_call.function.arguments)
        
        # if tool_name == "record_user_details":
        #     result = record_user_details(**arguments)
        # elif tool_name == "record_unknown_questions":
        #     result = record_unknown_questions(**arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role":"tool", "content": json.dumps(result), "tool_call_id": tool_call.id})
    return results

In [ ]:
reader = PdfReader("files/Profile1.pdf")
profile = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        profile += text + "\n"

with open("files/summary.txt", "r") as f:
    summary = f.read()

name = "Sabapathy Kathiresan"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any questions, use your record_unknown_questions tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career.\
If the user is engaging in a discussion, try to steer them towards getting in touch via email. Ask for their email and use the record_user_details tool to record their interest in getting in touch."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{profile}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    print(history)
    done = False
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        # If the LLM wants to call a tool, we do that!
        
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat).launch()